<a href="https://colab.research.google.com/github/atarasaki/DSWP-editados/blob/main/2020-11-25-LabDataChurn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## notebook para o Kaggle - construção em aula - 23/11/2020

In [ ]:
## importa pycaret
!pip install pycaret

     |████████████████████████████████| 256kB 9.3MB/s 
     |████████████████████████████████| 66.3MB 60kB/s 
     |████████████████████████████████| 1.8MB 46.3MB/s 
     |████████████████████████████████| 102kB 9.9MB/s 
     |████████████████████████████████| 6.8MB 43.6MB/s 
     |████████████████████████████████| 13.9MB 275kB/s 
     |████████████████████████████████| 1.6MB 48.7MB/s 
     |████████████████████████████████| 148.9MB 78kB/s 
     |████████████████████████████████| 276kB 52.1MB/s 
     |████████████████████████████████| 266kB 52.8MB/s 
     |████████████████████████████████| 174kB 54.1MB/s 
     |████████████████████████████████| 2.1MB 47.0MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 153kB 48.5MB/s 
     |████████████████████████████████| 163kB 50.9MB/s 
     |████████████████████████████████| 1.1MB 43.8MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 337kB 49.5MB/s 

In [ ]:
!pip install shap

     |████████████████████████████████| 327kB 11.6MB/s 
  Created wheel for shap: filename=shap-0.37.0-cp36-cp36m-linux_x86_64.whl size=463915 sha256=09396432bed8cd2a4232fd2775fe9695f8917e37df64433328a07a7efc52581f
  Stored in directory: /root/.cache/pip/wheels/df/ad/b0/aa7815ec68850d66551ef618095eccb962c8f6022f1d3dd989
Successfully built shap


In [ ]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

import seaborn as sns
sns.set_theme(style='ticks', palette='pastel')

from pycaret import regression, classification
import shap

from collections import Counter

import tensorflow as tf
from tensorflow import keras

In [ ]:
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


## Configurações de ambiente

In [ ]:
pd_od = pd.options.display

In [ ]:
pd_od.max_columns= 50
pd_od.max_rows = 50
pd_od.precision = 3

In [ ]:
# alternativa
# pd.set_option('display.precision',3)
#pd.set_option('display.max_columns',50)
# pd.set_option('display.max_rows',50)

## Facets - DataViz

In [ ]:
from IPython.core.display import display, HTML
import base64

In [ ]:
!pip install facets-overview==1.0.0

In [ ]:
from facets_overview.feature_statistics_generator import FeatureStatisticsGenerator

In [ ]:
# treino
url_T = 'https://raw.githubusercontent.com/atarasaki/DSWP-editados/main/Datasets/Labdata%20Churn%20Challenge%202020/train.csv'

# validação/teste
url_V = 'https://raw.githubusercontent.com/atarasaki/DSWP-editados/main/Datasets/Labdata%20Churn%20Challenge%202020/test.csv'

In [ ]:
df_T = pd.read_csv( url_T )

df_V = pd.read_csv( url_V)

In [ ]:
df_T.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,2560.1,0
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,332.65,0
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.7,0
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,1
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.8,1


In [ ]:
df_T.dtypes

id                    int64
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
dtype: object

In [ ]:
# Conversão de tipos : TotalCharges deveria ser float e está como object

df_T['TotalCharges2'] = pd.to_numeric( df_T['TotalCharges'], errors='coerce' )
df_T.dtypes

id                    int64
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                 int64
TotalCharges2       float64
dtype: object

In [ ]:
# aparentemente, conversão bem sucedida
df_T[['TotalCharges','TotalCharges2']].head()

,TotalCharges,TotalCharges2
0,2560.1,2560.10
1,332.65,332.65
2,6640.7,6640.70
3,3545.35,3545.35
4,1974.8,1974.80


In [ ]:
# índice 161 havia sido reportado na conversão pd.to_numeric() sem a opção errors='coerce'
df_T.loc[161,['TotalCharges','TotalCharges2']]

TotalCharges        
TotalCharges2    NaN
Name: 161, dtype: object

In [ ]:
# missing values nas colunas 'TotalCharges' e 'TotalCharges2'

df_T[['TotalCharges','TotalCharges2']].isnull().sum()

TotalCharges     0
TotalCharges2    8
dtype: int64

In [ ]:
# deleta 'TotalCharges'

df_T = df_T.drop( columns=['TotalCharges'], axis=1 )
df_T.head()

,id,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn,TotalCharges2
0,4030,Female,0,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,0,2560.10
1,6731,Male,0,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,0,332.65
2,6479,Female,0,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,0,6640.70
3,6861,Female,0,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,1,3545.35
4,3266,Male,0,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1,1974.80


In [ ]:
# 'SeniorCitizen' está como int64, mas deveria ser categórica
Counter( df_T['SeniorCitizen'] )

Counter({0: 4740, 1: 894})

In [ ]:
# alternativa
df_T['SeniorCitizen'].value_counts()

0    4740
1     894
Name: SeniorCitizen, dtype: int64

In [ ]:
# conveniente a nomenclatura Yes/No para o momento de criar dummy variables : nomes de coluna mais legíveis
d_SeniorCitizen = { 0 : 'No', 1 : 'Yes'}

In [ ]:
df_T['SeniorCitizen2'] = df_T['SeniorCitizen'].map( d_SeniorCitizen )
df_T['SeniorCitizen2'].head()

0    No
1    No
2    No
3    No
4    No
Name: SeniorCitizen2, dtype: object

In [ ]:
df_T.drop( columns=['SeniorCitizen'], inplace=True )
df_T.head()

,id,gender,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn,TotalCharges2,SeniorCitizen2
0,4030,Female,No,No,56.0,No,No phone service,DSL,Yes,Yes,Yes,Yes,No,No,Two year,No,NaN,45.05,0,2560.10,No
1,6731,Male,Yes,Yes,NaN,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,19.65,0,332.65,No
2,6479,Female,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,0,6640.70,No
3,6861,Female,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,1,3545.35,No
4,3266,Male,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1,1974.80,No


# listas por tipos de variáveis

In [ ]:
l_features_numericas = list( df_T.select_dtypes( [np.float64, np.int64] ) )
l_features_numericas.remove('Churn')
l_features_numericas

['id', 'tenure', 'MonthlyCharges', 'TotalCharges2']

In [ ]:
l_features_categoricas = list( df_T.select_dtypes( np.object ) )
l_features_categoricas

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'SeniorCitizen2']

# DataViz com Facets

In [ ]:
def mostra_dados( df ):
  fsg = FeatureStatisticsGenerator()
  dataframes = [ {'table': df, 'name': 'trainData'} ]
  censusProto = fsg.ProtoFromDataFrames(dataframes)
  protostr = base64.b64encode(censusProto.SerializeToString()).decode("utf-8")
  HTML_TEMPLATE = """<script src="https://cdnjs.cloudflare.com/ajax/libs/webcomponentsjs/1.3.3/webcomponents-lite.js"></script>
        <link rel="import" href="https://raw.githubusercontent.com/PAIR-code/facets/1.0.0/facets-dist/facets-jupyter.html">
        <facets-overview id="elem"></facets-overview>
        <script>
          document.querySelector("#elem").protoInput = "{protostr}";
        </script>"""
  html = HTML_TEMPLATE.format(protostr=protostr)
  display(HTML(html))

In [ ]:
mostra_dados( df_T )

# tratamento dos missing values

como trata os missing values ?

construir modelos preditivos para estimas os valores missing em em função das demais variáveis que não são missing

variáveis :
numéricas

* TotalCharges2 (0,14%)

* tenure (8,18%)


categóricas

* PaymentMethod (1,76%)

* Dependents (3,87%)


In [ ]:
# backup antes de transformações
df_T_org = df_T.copy()

In [ ]:
# restaura valores
#df_T = df_T_org

In [ ]:
df_T.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5634 entries, 0 to 5633
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                5634 non-null   int64  
 1   gender            5634 non-null   object 
 2   Partner           5634 non-null   object 
 3   Dependents        5416 non-null   object 
 4   tenure            5173 non-null   float64
 5   PhoneService      5634 non-null   object 
 6   MultipleLines     5634 non-null   object 
 7   InternetService   5634 non-null   object 
 8   OnlineSecurity    5634 non-null   object 
 9   OnlineBackup      5634 non-null   object 
 10  DeviceProtection  5634 non-null   object 
 11  TechSupport       5634 non-null   object 
 12  StreamingTV       5634 non-null   object 
 13  StreamingMovies   5634 non-null   object 
 14  Contract          5634 non-null   object 
 15  PaperlessBilling  5634 non-null   object 
 16  PaymentMethod     5535 non-null   object 


In [ ]:
# função para construir modelos ML para estimar os missing values

def prepara_dataframes( df, target, l_colunas_mv ):
  nome_df2 = 'df' + '_' + target + '_sem_mv' # constrói nome de um novo dataframe
  print( nome_df2 )

  l_colunas_mv.remove( target )
  print( f'Features ignoradas : {l_colunas_mv}')

  df2 = df.copy()
  df3 = df.copy()

  df2 = df2.dropna() # exclui todas as linhas com missing values
  df2 = df2.drop( columns=['Churn'], axis=1 )

  # para reduzir o bias, drop features que são missing values
  df2 = df2.drop( columns=l_colunas_mv, axis=1 )
  print( df2.isna().sum() )

  # apontar os missing values
  df3[target + '_mv'] = np.where( df3[target].isna(), 1, 0)

  return ( df2, df3 )

In [ ]:
# colunas com missing values
l_colunas_mv = ['TotalCharges2','tenure','PaymentMethod','Dependents']

# l_colunas_mv é modificada pela função 'prepara_dataframes()'
l_colunas_loop = l_colunas_mv.copy()

# trata as variáveis em loop : 'TotalCharges2', 'tenure', 'PaymentMethod', 'Dependents'

for target in l_colunas_loop:
  nome_df2 = 'df' + '_' + target + '_sem_mv'
  print( nome_df2 )
  vars()[nome_df2], df_T = prepara_dataframes( df_T, target, l_colunas_mv )

df_TotalCharges2_sem_mv
df_TotalCharges2_sem_mv
Features ignoradas : ['tenure', 'PaymentMethod', 'Dependents']
id                  0
gender              0
Partner             0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges2       0
SeniorCitizen2      0
dtype: int64
df_tenure_sem_mv
df_tenure_sem_mv
Features ignoradas : ['PaymentMethod', 'Dependents']
id                  0
gender              0
Partner             0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
MonthlyCharges      0
TotalCharges2       0
SeniorCitizen2      0
TotalCharges2_mv   

In [ ]:
df_TotalCharges2_sem_mv

,id,gender,Partner,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges2,SeniorCitizen2
2,6479,Female,Yes,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,110.80,6640.70,No
3,6861,Female,No,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,101.90,3545.35,No
4,3266,Male,Yes,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,70.75,1974.80,No
5,6368,Male,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,48.40,3442.80,No
6,5830,Male,Yes,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,59.95,1848.80,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5628,946,Female,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,No,74.40,215.80,No
5629,6684,Male,No,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,One year,No,44.15,1931.30,No
5631,4803,Female,No,Yes,No,DSL,No,Yes,No,Yes,No,No,Month-to-month,No,54.65,1517.50,No
5632,6317,Male,Yes,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,25.25,1573.05,No


In [ ]:
df_tenure_sem_mv

,id,gender,Partner,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges2,SeniorCitizen2,TotalCharges2_mv
2,6479,Female,Yes,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,110.80,6640.70,No,0
3,6861,Female,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,101.90,3545.35,No,0
4,3266,Male,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,70.75,1974.80,No,0
5,6368,Male,No,70.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,48.40,3442.80,No,0
6,5830,Male,Yes,31.0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,59.95,1848.80,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5628,946,Female,No,3.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,No,74.40,215.80,No,0
5629,6684,Male,No,43.0,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,One year,No,44.15,1931.30,No,0
5631,4803,Female,No,28.0,Yes,No,DSL,No,Yes,No,Yes,No,No,Month-to-month,No,54.65,1517.50,No,0
5632,6317,Male,Yes,63.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,25.25,1573.05,No,0


In [ ]:
df_PaymentMethod_sem_mv

,id,gender,Partner,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges2,SeniorCitizen2,TotalCharges2_mv,tenure_mv
2,6479,Female,Yes,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,No,0,0
3,6861,Female,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,No,0,0
4,3266,Male,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,No,0,0
5,6368,Male,No,70.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,Bank transfer (automatic),48.40,3442.80,No,0,0
6,5830,Male,Yes,31.0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,Electronic check,59.95,1848.80,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5628,946,Female,No,3.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),74.40,215.80,No,0,0
5629,6684,Male,No,43.0,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,One year,No,Electronic check,44.15,1931.30,No,0,0
5631,4803,Female,No,28.0,Yes,No,DSL,No,Yes,No,Yes,No,No,Month-to-month,No,Bank transfer (automatic),54.65,1517.50,No,0,0
5632,6317,Male,Yes,63.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.25,1573.05,No,0,0


In [ ]:
df_Dependents_sem_mv

,id,gender,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges2,SeniorCitizen2,TotalCharges2_mv,tenure_mv,PaymentMethod_mv
2,6479,Female,Yes,No,60.0,Yes,Yes,Fiber optic,No,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),110.80,6640.70,No,0,0,0
3,6861,Female,No,No,37.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,No,0,0,0
4,3266,Male,Yes,Yes,29.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.75,1974.80,No,0,0,0
5,6368,Male,No,No,70.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,Yes,Two year,No,Bank transfer (automatic),48.40,3442.80,No,0,0,0
6,5830,Male,Yes,Yes,31.0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,No,Electronic check,59.95,1848.80,No,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5628,946,Female,No,No,3.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),74.40,215.80,No,0,0,0
5629,6684,Male,No,No,43.0,No,No phone service,DSL,No,No,Yes,Yes,No,Yes,One year,No,Electronic check,44.15,1931.30,No,0,0,0
5631,4803,Female,No,No,28.0,Yes,No,DSL,No,Yes,No,Yes,No,No,Month-to-month,No,Bank transfer (automatic),54.65,1517.50,No,0,0,0
5632,6317,Male,Yes,Yes,63.0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.25,1573.05,No,0,0,0
